## Modeling Exercises

In [1]:
# for presentation purposes
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# visualize 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# wrangle
import acquire as a
import prepare as p

# working with dates
from datetime import datetime

# to evaluated performance using rmse
from sklearn.metrics import mean_squared_error
from math import sqrt 

# for tsa 
import statsmodels.api as sm

# holt's linear trend model. 
from statsmodels.tsa.api import Holt, ExponentialSmoothing

import statsmodels.api as sm

In [2]:
# Get data from the URL
url = "https://ds.codeup.com/saas.csv"

df = pd.read_csv(url)
df.head()


,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


### Get the Data

In [4]:
#Get data from the local csv
df = pd.read_csv('saas.csv')

In [5]:
df.head()

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


### Prep the Data

In [7]:
#Rename the columns so they dont have capitalization
df = df.rename(columns = {'Month_Invoiced':'month_invoiced', 'Customer_Id':'customer_id',
                              'Invoice_Id':'invoice_id', 'Subscription_Type':'subscription_type',
                    'Amount':'amount'})

# Reassign the month invoiced column to be a datetime type
df.month_invoiced = pd.to_datetime(df.month_invoiced)

# Set the index as that date and then sort index (by the date)
df = df.set_index("month_invoiced").sort_index()

In [10]:
df.index
#4 years of data by month

DatetimeIndex(['2014-01-31', '2014-01-31', '2014-01-31', '2014-01-31',
               '2014-01-31', '2014-01-31', '2014-01-31', '2014-01-31',
               '2014-01-31', '2014-01-31',
               ...
               '2017-12-31', '2017-12-31', '2017-12-31', '2017-12-31',
               '2017-12-31', '2017-12-31', '2017-12-31', '2017-12-31',
               '2017-12-31', '2017-12-31'],
              dtype='datetime64[ns]', name='month_invoiced', length=790458, freq=None)

In [ ]:
#Change subscription type data type

In [8]:
#Check the data out
df.head(2)

,customer_id,invoice_id,subscription_type,amount
month_invoiced,,,,
2014-01-31,1000000,5000000,0.0,0.0
2014-01-31,1000001,5000001,2.0,10.0


In [9]:
#Look at the data
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 790458 entries, 2014-01-31 to 2017-12-31
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_id        790458 non-null  int64  
 1   invoice_id         790458 non-null  int64  
 2   subscription_type  790458 non-null  float64
 3   amount             790458 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 30.2 MB


In [11]:
#Check for nulls (there are none)
df.isnull().sum()

customer_id          0
invoice_id           0
subscription_type    0
amount               0
dtype: int64

In [ ]:
#Sum the amount 
#Subscription type is * 5 amount

### 1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum.